In [0]:
%sql
USE CATALOG mvp;
USE SCHEMA bronze;

In [0]:
volume_path = "/Volumes/mvp/staging/anac/txt"
files = dbutils.fs.ls(volume_path)
df_spark = spark.read.option("header", "true").option("sep", ";").csv(files[0].path.replace("dbfs:", ""))
colunas_inicial = df_spark.columns
for file in files[1:]:
    txt_path = file.path.replace("dbfs:", "")
    df = spark.read.option("header", "true").option("sep", ";").csv(txt_path)
    colunas = df.columns
    if (colunas != colunas_inicial):
        print("Arquivo com colunas diferentes: " + txt_path)
        continue
    df_spark = df_spark.union(df)
    print("Arquivo processado: " + txt_path)
print("-"*20)
print("Total de linhas: " + str(df_spark.count()))
display(df_spark.limit(10))

In [0]:
df_spark.write.format("delta").mode("overwrite").saveAsTable("voos")